# Step 1: prepare data from JH about confirmed cases

We need to get the approximate growth rate of the number of confirmed case for each country. This returns a dataset of all countries with more than 250 cases to this date and their respective growth rate. We will use this as training data

In [705]:
#STEP1: prepare data from JH about confirmed cases
#import data
import pandas as pd
import numpy as np
URL1 = "https://raw.githubusercontent.com/AlexOktay/covid19/master/time_series_covid19_confirmed_global.csv"
country_data = pd.read_csv(URL1)

#Drop LAT and LONG columns, rename "Country/Region"
country_data = country_data.drop(["Lat", "Long"], axis = 1)
country_data.rename(columns={"Country/Region":"Countries"}, inplace=True)

#Agregate data from provinces to countries
dates = list(country_data.columns)
country_data = country_data[dates].groupby(country_data["Countries"]).sum()

# we want to see a daily growth rate for number of cases (assumption: exponential)
# We first need to see when the epidemic started spreading in each country
spread_start = pd.DataFrame(columns=["Country", "Start"])
for index, row in country_data.iterrows():
  for date in country_data:
    if country_data.loc[index, date] > 0:
      spread_start = spread_start.append({"Country":index, "Start":date}, ignore_index=True)
      break  
    else:
      pass

# exponential growth is based on equation y(t) = a__e^rt for a the initial value,
# y(t) the current value, r the growth rate and t the number of days since start of pandemic
# if we resolve for r: r = ln(y(t)/a) / t 
# for simplification purposes, we estimate a=1 (there is only one initial patient)
def rate(y, t):
  "Calculate the exponantial rate of growth"
  growth = np.log(y) / t
  return growth

# Create a new column with growth rate for each country
country_data.insert(0, "Growth_rate",0.00)
last_update = country_data.columns[-1]
for index, row in country_data.iterrows():
  temp = 0
  start_date = spread_start.iloc[temp,1]
  t = len(country_data.columns) - (country_data.columns.get_loc(start_date)+1)
  y = country_data.loc[index, last_update]
  country_data.at[index, 'Growth_rate'] = rate(y,t)
  temp = temp+1

# for training, we will only use countries whcih already have a heavy number
# of confirmed cases (here we set the threshold at >250). We will try to predict
# growth rate for countries under 250 
country_data_copy = country_data #useful in step 4
for index, row in country_data.iterrows():
  if country_data.loc[index, last_update] < 250:
    country_data = country_data.drop(index)
  else:
    pass

# finaly, drop columns with the exact evolution of confirmed cases: we will be 
#working only on the growth_rate to try to predict it
country_data.drop(country_data.iloc[:, 1:], axis=1, inplace=True)

country_data

,Growth_rate
Countries,
Algeria,0.187929
Andorra,0.174602
Argentina,0.199326
Armenia,0.181127
Australia,0.251654
Austria,0.279480
Bahrain,0.192006
Belgium,0.277920
Brazil,0.254266


# Step 2: Set the economic features
We will use economic data to try to predict the actual growth rate of the epidemic in countries. This will be the features used in our model. This returns a table where each row = one heavily infected country with their respective economic features (columns)

We uses two databases for this:
- general_data for basic features about population (population, density...)
- bank_data for specific economic indicators (from World Bank)

## Step 2.1 First data source: general_data

In [706]:
# import the data
URL2 = "https://raw.githubusercontent.com/AlexOktay/covid19/master/countries%20of%20the%20world.csv"
general_data = pd.read_csv(URL2)

#Remove space at the end of the country name in general data, and edit names to match country_data
for index, row in general_data.iterrows():  
  general_data.iloc[index, 0] = general_data.iloc[index, 0].rstrip()
general_data.Country.replace({"United States":"US"}, inplace=True)
general_data.Country.replace({"Czech Republic":"Czechia"}, inplace=True)
general_data.Country.replace({"Taiwan":"Taiwan*"}, inplace=True)

# merge general data with epidemic data using SQL
from sqlalchemy import create_engine
db = create_engine('sqlite://', echo=False)
country_data.to_sql("country", con=db)
general_data.to_sql("general", con=db) 

query = """ 
SELECT *
FROM country c
INNER JOIN general g
ON c.countries=g.country
""" 
merge_data1 = pd.read_sql(query, con=db) 

#Drop useless columns and columns that do not seem relevant
merge_data1.drop(['Country','Region', 'index', 'Coastline (coast/area ratio)', 'Other (%)', 'Climate'],axis=1,inplace=True)
merge_data1.dtypes

#Convert columns to float
to_replace = ["Pop. Density (per sq. mi.)","Net migration","Infant mortality (per 1000 births)","Literacy (%)", 
              "Phones (per 1000)", "Arable (%)","Crops (%)","Birthrate","Deathrate", "Agriculture", "Industry", "Service"]
for i in to_replace:
  merge_data1[i] = merge_data1[i].str.replace(',', '.')

cols = merge_data1.columns.drop('Countries')
merge_data1[cols] = merge_data1[cols].apply(pd.to_numeric, errors='coerce')

# Drop rows with NaN (only 2 of them so no efficiency loss)
merge_data1 = merge_data1.dropna()

merge_data1

,Countries,Growth_rate,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Birthrate,Deathrate,Agriculture,Industry,Service
0,Algeria,0.187929,32930091,2381740,13.8,-0.39,31.00,6000.0,70.0,78.1,3.22,0.25,17.14,4.61,0.101,0.600,0.298
2,Argentina,0.199326,39921833,2766890,14.4,0.61,15.18,11200.0,97.1,220.4,12.31,0.48,16.73,7.55,0.095,0.358,0.547
3,Armenia,0.181127,2976372,29800,99.9,-6.47,23.28,3500.0,98.6,195.7,17.55,2.30,12.07,8.23,0.239,0.343,0.418
4,Australia,0.251654,20264082,7686850,2.6,3.98,4.69,29000.0,100.0,565.5,6.55,0.04,12.14,7.51,0.038,0.262,0.700
5,Austria,0.279480,8192880,83870,97.7,2.00,4.66,30000.0,98.0,452.2,16.91,0.86,8.74,9.76,0.018,0.304,0.678
6,Bahrain,0.192006,698585,665,1050.5,1.05,17.27,16900.0,89.1,281.3,2.82,5.63,17.80,4.14,0.005,0.387,0.608
7,Belgium,0.277920,10379067,30528,340.0,1.23,4.68,29100.0,98.0,462.6,23.28,0.40,10.38,10.27,0.010,0.240,0.749
8,Brazil,0.254266,188078227,8511965,22.1,-0.03,29.61,7600.0,86.4,225.3,6.96,0.90,16.56,6.17,0.084,0.400,0.516
9,Bulgaria,0.177505,7385367,110910,66.6,-4.58,20.55,7600.0,98.6,336.3,40.02,1.92,9.65,14.27,0.093,0.304,0.603
10,Canada,0.264109,33098932,9984670,3.3,5.96,4.75,29800.0,97.0,552.2,4.96,0.02,10.78,7.80,0.022,0.294,0.684


## Step 2.2 World bank data


In [707]:
URL2 = "https://raw.githubusercontent.com/AlexOktay/covid19/master/API_3_DS2_en_csv_v2_896475.csv"
economics = pd.read_csv(URL2)

#Data cleaning
economics = economics.drop([0,1,2,3], axis=0)
economics = economics.pivot(index='Data Source', columns='Unnamed: 2', values='Unnamed: 4')
economics = economics.rename_axis('Country_Name')

rename_countries={'Congo, Dem. Rep.':'Congo (Kinshasa)', 'Congo, Rep.':'Congo (Brazzaville)', 'Korea, Rep.':'Korea, South',
                  'St. Kitts and Nevis':'Saint Kitts and Nevis', 'St. Lucia':'Saint Lucia',
                  'St. Vincent and the Grenadines':'Saint Vincent and the Grenadines', 'United States':'US', 'Czech Republic':'Czechia',
                  'Egypt, Arab Rep.':'Egypt', 'Iran, Islamic Rep.':'Iran', 'Russian Federation':'Russia'}

economics = economics.rename(index=rename_countries)
merge_data1.to_sql("source", con=db)
economics.to_sql("econ", con=db)

query = """ 
SELECT *
FROM source s
INNER JOIN econ e
ON e.Country_name = s.countries
""" 
merge_data2 = pd.read_sql(query, con=db) 
#Drop useless columns and columns that do not seem relevant
merge_data2 = merge_data2[['Countries', 'Growth_rate', 'Population', 'Area (sq. mi.)', 'Pop. Density (per sq. mi.)', 'Net migration',
       'Infant mortality (per 1000 births)', 'GDP ($ per capita)', 'Literacy (%)', 'Phones (per 1000)','Arable (%)','Crops (%)','Birthrate','Deathrate',
       'Agriculture','Industry','Service', 'Adjusted net national income per capita (constant 2010 US$)','Current account balance (% of GDP)',
       'Exports of goods and services (% of GDP)','External debt stocks (% of GNI)','Foreign direct investment, net inflows (% of GDP)',
       'Foreign direct investment, net outflows (% of GDP)','Gross capital formation (% of GDP)','Gross domestic savings (% of GDP)',
       'Imports of goods and services (% of GDP)','Inflation, consumer prices (annual %)','Short-term debt (% of total reserves)','Total debt service (% of GNI)','Trade (% of GDP)']]
merge_data2

,Countries,Growth_rate,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Birthrate,Deathrate,Agriculture,Industry,Service,Adjusted net national income per capita (constant 2010 US$),Current account balance (% of GDP),Exports of goods and services (% of GDP),External debt stocks (% of GNI),"Foreign direct investment, net inflows (% of GDP)","Foreign direct investment, net outflows (% of GDP)",Gross capital formation (% of GDP),Gross domestic savings (% of GDP),Imports of goods and services (% of GDP),"Inflation, consumer prices (annual %)",Short-term debt (% of total reserves),Total debt service (% of GNI),Trade (% of GDP)
0,Algeria,0.187929,32930091,2381740,13.8,-0.39,31.00,6000.0,70.0,78.1,3.22,0.25,17.14,4.61,0.101,0.600,0.298,3651.397735,-13.178013,22.661204,3.459552,0.717464,-0.002017,48.047998,37.492329,33.217950,5.591116,1.999293,0.145520,55.879155
1,Argentina,0.199326,39921833,2766890,14.4,0.61,15.18,11200.0,97.1,220.4,12.31,0.48,16.73,7.55,0.095,0.358,0.547,9110.293202,-4.916457,11.242721,37.756505,1.791961,0.179810,18.757232,16.019342,13.980611,NaN,100.264613,6.374368,25.223332
2,Armenia,0.181127,2976372,29800,99.9,-6.47,23.28,3500.0,98.6,195.7,17.55,2.30,12.07,8.23,0.239,0.343,0.418,3131.939101,-2.988021,37.329503,86.138062,2.176847,0.252066,19.289686,7.651455,49.500638,0.969553,38.416431,12.112162,86.830141
3,Australia,0.251654,20264082,7686850,2.6,3.98,4.69,29000.0,100.0,565.5,6.55,0.04,12.14,7.51,0.038,0.262,0.700,42703.690700,-2.694728,21.193200,NaN,3.574156,0.626482,24.070736,24.687193,20.576800,1.948647,NaN,NaN,41.770000
4,Austria,0.279480,8192880,83870,97.7,2.00,4.66,30000.0,98.0,452.2,16.91,0.86,8.74,9.76,0.018,0.304,0.678,39821.386540,1.538314,54.037671,NaN,3.240018,2.422403,24.765517,28.291363,50.735505,2.081269,NaN,NaN,104.773176
5,Bahrain,0.192006,698585,665,1050.5,1.05,17.27,16900.0,89.1,281.3,2.82,5.63,17.80,4.14,0.005,0.387,0.608,17670.822980,-4.514929,75.444974,NaN,1.464419,0.646266,32.942146,41.375428,67.385040,1.387344,NaN,NaN,142.830014
6,Belgium,0.277920,10379067,30528,340.0,1.23,4.68,29100.0,98.0,462.6,23.28,0.40,10.38,10.27,0.010,0.240,0.749,37822.395280,1.228688,82.311445,NaN,-7.185446,-1.260401,24.113746,25.507449,80.917742,2.125971,NaN,NaN,163.229188
7,Brazil,0.254266,188078227,8511965,22.1,-0.03,29.61,7600.0,86.4,225.3,6.96,0.90,16.56,6.17,0.084,0.400,0.516,9363.804986,-0.731134,12.573463,26.939831,3.354365,1.039178,15.035636,16.038056,11.571028,3.446373,14.014606,4.734802,24.144490
8,Bulgaria,0.177505,7385367,110910,66.6,-4.58,20.55,7600.0,98.6,336.3,40.02,1.92,9.65,14.27,0.093,0.304,0.603,7425.480447,3.644649,68.146473,68.556977,3.470124,0.877055,20.138753,23.276245,63.756637,2.064355,33.606064,14.783630,131.903110
9,Canada,0.264109,33098932,9984670,3.3,5.96,4.75,29800.0,97.0,552.2,4.96,0.02,10.78,7.80,0.022,0.294,0.684,41240.163140,-2.816305,31.463120,NaN,1.767587,4.881487,23.571527,21.191067,33.679512,1.596884,NaN,NaN,65.142632


# Step 3: Create and fit the model
Build our model using ML

In [708]:
#With XGBoost

#Preparation
data = merge_data2
y = data.Growth_rate
data_features = list(data.columns)
data_features.remove("Countries")
data_features.remove("Growth_rate")
X=data[data_features]

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split (X, y, train_size=0.99, test_size=0.01, random_state=5)
#With train size = 0.85, average mean absolute error is approx 0.03

#Fit model and mpute missing data with median
from xgboost import XGBRegressor
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer(strategy='median')
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(X_train))
imputed_X_valid = pd.DataFrame(my_imputer.transform(X_valid))

imputed_X_train.columns = X_train.columns
imputed_X_valid.columns = X_valid.columns

model = XGBRegressor(n_estimators=1000, learning_rate=0.04)
model.fit(X_train, y_train, 
	          early_stopping_rounds=10, 
	          eval_set=[(X_valid, y_valid)],
            verbose=False)

#Prediction test
from sklearn.metrics import mean_absolute_error
predictions = model.predict(X_valid)
print("Mean Absolute Error: " + str(mean_absolute_error(predictions, y_valid)))

[19:56:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Mean Absolute Error: 0.0866666227224076


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


# Step 4: predict data using model

## Step 4.1: data preparation

In [709]:
# Gather the list of countries we didn't use for training
for index, row in country_data_copy.iterrows():
  if country_data_copy.loc[index, last_update] > 250:
    country_data_copy = country_data_copy.drop(index)
  else:
    pass
country_data_copy.drop(country_data_copy.iloc[:, 0:], axis=1, inplace=True)
general_data.Country.replace({"Antigua & Barbuda":"Antigua and Barbuda"}, inplace=True)
general_data.Country.replace({"Bahamas, The":"Bahamas"}, inplace=True)
general_data.Country.replace({"Bosnia & Herzegovina":"Bosnia and Herzegovina"}, inplace=True)
general_data.Country.replace({"Cape Verde":"Cabo Verde"}, inplace=True)
general_data.Country.replace({"Central African Rep.":"Central African Republic"}, inplace=True)
general_data.Country.replace({"Congo, Repub. of the, The":"Congo (Brazzaville)"}, inplace=True)
general_data.Country.replace({"Congo, Dem. Rep, The":"Congo (Kinshasa)"}, inplace=True)
general_data.Country.replace({"Gambia, The":"Gambia"}, inplace=True)
general_data.Country.replace({"Macedonia":"North Macedonia"}, inplace=True)
general_data.Country.replace({"Saint Kitts & Nevis":"Saint Kitts and Nevis"}, inplace=True)
general_data.Country.replace({"East Timor":"Timor-Leste"}, inplace=True)
general_data.Country.replace({"Trinidad & Tobago":"Trinidad and Tobago"}, inplace=True)
general_data.Country.replace({"Gaza Strip":"West Bank and Gaza"}, inplace=True)
#Countries left out due to missing data: Eswatini, Holy See, Kosovo, Montenegro

# Join with required data, and do the data cleaning as done in step 2
country_data_copy.to_sql("pred", con=db)
general_data.to_sql("gen", con=db)

query = """ 
SELECT *
FROM pred c
INNER JOIN gen g
ON c.countries=g.country
INNER JOIN econ e
ON e.Country_name = c.countries
""" 
prediction_data = pd.read_sql(query, con=db) 
prediction_data.drop(['Country','Region', 'index', 'Coastline (coast/area ratio)', 'Other (%)', 'Climate'],axis=1,inplace=True)
prediction_data = prediction_data[['Countries', 'Population', 'Area (sq. mi.)', 'Pop. Density (per sq. mi.)', 'Net migration',
       'Infant mortality (per 1000 births)', 'GDP ($ per capita)', 'Literacy (%)', 'Phones (per 1000)','Arable (%)','Crops (%)','Birthrate','Deathrate',
       'Agriculture','Industry','Service', 'Adjusted net national income per capita (constant 2010 US$)','Current account balance (% of GDP)',
       'Exports of goods and services (% of GDP)','External debt stocks (% of GNI)','Foreign direct investment, net inflows (% of GDP)',
       'Foreign direct investment, net outflows (% of GDP)','Gross capital formation (% of GDP)','Gross domestic savings (% of GDP)',
       'Imports of goods and services (% of GDP)','Inflation, consumer prices (annual %)','Short-term debt (% of total reserves)','Total debt service (% of GNI)','Trade (% of GDP)']]

to_replace = ["Pop. Density (per sq. mi.)","Net migration","Infant mortality (per 1000 births)","Literacy (%)", 
              "Phones (per 1000)", "Arable (%)","Crops (%)","Birthrate","Deathrate", "Agriculture", "Industry", "Service"]
for i in to_replace:
  prediction_data[i] = prediction_data[i].str.replace(',', '.')

cols = prediction_data.columns.drop('Countries')
prediction_data[cols] = prediction_data[cols].apply(pd.to_numeric, errors='coerce')

country_list_pred = prediction_data["Countries"]
prediction_data.drop(['Countries'], axis=1, inplace=True)

prediction_data

,Population,Area (sq. mi.),Pop. Density (per sq. mi.),Net migration,Infant mortality (per 1000 births),GDP ($ per capita),Literacy (%),Phones (per 1000),Arable (%),Crops (%),Birthrate,Deathrate,Agriculture,Industry,Service,Adjusted net national income per capita (constant 2010 US$),Current account balance (% of GDP),Exports of goods and services (% of GDP),External debt stocks (% of GNI),"Foreign direct investment, net inflows (% of GDP)","Foreign direct investment, net outflows (% of GDP)",Gross capital formation (% of GDP),Gross domestic savings (% of GDP),Imports of goods and services (% of GDP),"Inflation, consumer prices (annual %)",Short-term debt (% of total reserves),Total debt service (% of GNI),Trade (% of GDP)
0,31056997,647500,48.0,23.06,163.07,700.0,36.0,3.2,12.13,0.22,46.60,20.34,0.380,0.240,0.380,NaN,-21.430835,5.904816,13.389900,0.255222,0.055769,19.174225,7.155705,45.332065,4.975952,5.366420,0.313625,51.236881
1,3581655,28748,124.6,-4.93,21.52,4500.0,86.5,71.2,21.09,4.42,15.11,5.22,0.232,0.188,0.579,4226.515631,-7.541137,31.556646,77.061501,7.852228,-0.815168,25.049155,8.791835,46.604666,1.986661,59.884852,3.924523,78.161312
2,12127071,1246700,9.7,0.00,191.19,1900.0,42.0,7.8,2.41,0.24,45.11,24.20,0.096,0.658,0.246,1650.887960,-0.518218,29.004100,44.445771,-6.057209,1.107077,24.130305,29.881683,23.252721,31.691686,21.168801,7.754783,52.256821
3,69108,443,156.0,-6.15,19.46,11000.0,89.0,549.9,18.18,4.55,16.93,5.37,0.038,0.220,0.743,NaN,-8.846589,NaN,NaN,10.690529,-0.159342,NaN,NaN,NaN,2.432488,NaN,NaN,NaN
4,7961619,86600,91.9,-4.90,81.74,3400.0,97.0,137.1,19.63,2.71,20.74,9.75,0.141,0.457,0.402,NaN,4.122197,48.547865,39.553139,7.016879,6.275164,24.378926,31.072197,41.854451,12.904891,10.099674,5.821852,90.402316
5,147365352,144000,1023.4,-0.71,62.60,1900.0,43.1,7.3,62.11,3.07,29.80,8.27,0.199,0.198,0.603,1053.243258,-2.396644,15.036108,18.106186,0.724959,0.050753,30.510455,25.329251,20.267892,5.702070,32.205792,0.832432,35.304000
6,279912,431,649.5,-0.31,12.50,15700.0,97.4,481.9,37.21,2.33,12.71,8.67,0.060,0.160,0.780,NaN,NaN,42.094349,NaN,5.735191,5.735191,10.946510,6.005825,40.628848,4.660185,NaN,NaN,82.723197
7,10293011,207600,49.6,2.54,13.37,6100.0,99.6,319.1,29.55,0.60,11.16,14.02,0.093,0.316,0.591,6295.316666,-1.740287,66.789600,75.156544,2.332139,0.123889,28.030359,29.891194,66.578155,6.031837,132.936177,8.370185,133.367755
8,287730,22966,12.5,0.00,25.69,4900.0,94.1,115.7,2.85,1.71,28.84,5.72,0.142,0.152,0.612,3437.911494,-7.126376,54.976482,80.136921,1.395906,0.017550,18.693066,13.793685,58.435017,NaN,1.538907,6.017018,113.411499
9,7862944,112620,69.8,0.00,85.00,1100.0,40.9,9.7,18.08,2.40,38.85,12.22,0.316,0.138,0.546,694.332581,-10.003729,27.286400,31.565290,2.161977,0.191366,28.380186,10.840847,40.290729,0.079071,NaN,1.146442,67.577128


## Step 4.2: prediction

In [718]:
#Make prediction
prediction_values = model.predict(prediction_data)
final_prediction = pd.DataFrame({"Countries": country_list_pred})
final_prediction["Estimated exp growth rate"]=prediction_values
final_prediction.set_index("Countries", inplace=True)
final_prediction.sort_values(by="Estimated exp growth rate", inplace=True, ascending=False)

#Uncomment to download results
#final_prediction.to_csv("results.csv", sep=",")
#from google.colab import files
#files.download('results.csv')

final_prediction

,Estimated exp growth rate
Countries,
Monaco,0.276902
Cuba,0.258761
Somalia,0.251194
Chad,0.246432
Eritrea,0.241888
San Marino,0.234887
Central African Republic,0.234673
Kazakhstan,0.232438
Gabon,0.232339
